In [1]:
import pandas as pd
df = pd.read_csv('datasets/POS_TRANSACTIONS_2018 .csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459258 entries, 0 to 459257
Data columns (total 5 columns):
Location            459258 non-null int64
Transaction_Id      459258 non-null int64
Transaction_Date    459258 non-null object
Product_Name        459258 non-null object
Quantity            459258 non-null int64
dtypes: int64(3), object(2)
memory usage: 17.5+ MB


In [2]:
print('----Location----')
print(df['Location'].value_counts())
print('-------Transaction_Id-------')
print(df['Transaction_Id'])
print('----Transaction_Date----')
print(df['Transaction_Date'].value_counts())
print('----Product_Name----')
print(df['Product_Name'].value_counts())
print('----Quantity----')
print(df['Quantity'].value_counts())

----Location----
7     46570
3     46381
6     46299
2     46100
5     46032
8     45993
1     45896
10    45516
4     45354
9     45117
Name: Location, dtype: int64
-------Transaction_Id-------
0           12359
1           12362
2           12362
3           12365
4           12371
5           12380
6           12383
7           12383
8           12386
9           12386
10          12392
11          12392
12          12401
13          12401
14          12401
15          12401
16          12404
17          12410
18          12413
19          12470
20          12473
21          12476
22          12482
23          12563
24          12563
25          12563
26          12563
27          12563
28          12572
29          12572
           ...   
459228    1221800
459229    1221800
459230    1221803
459231    1221809
459232    1221827
459233    1221827
459234    1221830
459235    1221833
459236    1221833
459237    1221839
459238    1221842
459239    1221842
459240    1221845
459241    122